In [1]:
#%% Conectar con google drive, donde se encuentran los datos requeridos

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
#%% Se montan las librerías base para todo el proceso

import os
import sys
import pickle
import pandas as pd
import numpy as np
from os.path import expanduser, join

In [3]:
#%% Se cargan las rutas donde se han de leer y guardar los resultados

# home es la ruta principal del drive donde se almacena toda la informacion de este trabajo
home = expanduser('/gdrive/Shareddrives/DiegoHeredia-Econofisica /Proyecto')

# home_predata hace referencia a los datos proporcionados antes de ser filtrados y reestructurados
home_predata = join(home, 'predatos_time_series_RR')

# home_data hace referencia a los datos de los sujetos que se usaran para el analisis
home_data = join(home, 'datos_time_series_RR')

In [8]:
''' Se leen los datos de la carpeta 'predatos_time_series_RR' y se generan en 'datos_time_series_RR'
    tres diccionarios por sujeto y por altitud (A,B,C) que tengan las series de tiempo para los estados de
    Pre-siesta (PRE), Sueño-N2 (M2) y Post-siesta (POS) '''

# filtros es un diccionario que relaciona el formato de los archivos con la altitud a la que se tomaron las medidas
filtros = {'A': '0000m snm', 'B': '2600m snm', 'C':'4000m snm'}

# por cada filtro se extraen los diccionarios de los sujetos en los tres estados de sueño y vigilia
for filtro in filtros:

  # se carga la lista de todos los archivos posibles antes de ser organizados y filtrados
  filelist = os.listdir(home_predata)

  # se define un diccionario donde por cada sujeto se guardaran los nombres de los archivos que le corresponden
  predictionary = {}

  # se itera sobre todos los archivos en 'predata'
  for x in filelist:

      ''' Del formato de los archivos, por ejemplo 'F05C POS.txt', solo se extrae la informacion del sujeto y su altitud (e.g. 'F05C')
          para poder agrupar por estado Pre-siesta (PRE), Sueño-N2 (M2) y Post-siesta (POS) '''

      key = x[0:len('####')]

      ''' si el archivo corresponde al filtro o la altitud que se requiere, entonces su nombre,
          por ejemplo 'F05C POS.txt', se almacena en predictionary bajo la categoria de 'F05C' '''

      if x[len('###')] == filtro:
        if x[len('#####'):] == 'PRE.txt' or x[len('#####'):] == 'N2.txt' or x[len('#####'):] == 'POS.txt':

          group = predictionary.get(key,[])
          group.append(x)
          predictionary[key] = group

  ''' predictionary sera de la forma {'F07B': ['F07B PRE.txt', 'F07B N2.txt', 'F07B POS.txt'], 'F08B': ['F08B PRE.txt', 'F08B POS.txt'], ...} 
      lo que sigue entonces es filtrar los datos para dejar aquellos que tengan informacion de los tres estados de sueño y vigilia '''

  # se define un diccionario donde se dejaran los nombres de los archivos ya filtrados y listos para crear los diccionarios de las series de tiempo
  dictionary = {}

  # se itera sobre los sujetos en predictionary
  for sujeto in predictionary:

    # se verifica que los sujetos tengan los datos de los tres estados, y se exluye 'F08' para que cada altitud quede con la misma poblacion de sujetos
    if len(predictionary[sujeto]) == 3 and sujeto != 'F08' + filtro:
      dictionary[sujeto] = predictionary[sujeto]
      dictionary[sujeto].sort()

  # se guarda el diccionario con los nombres de los archivos ya filtrados   
  fdic = join(home, filtro + '_dict_sujetos_data_completa.npy')
  #np.save(fdic, dictionary)

  # se guarda una lista de sujetos con los datos completos
  fdic = join(home, filtro + '_sujetos_data_completa.npy')
  #np.save(fdic, list(dictionary.keys()))

  ''' Con el diccionario de los nombres de los archivos ya filtrados, se guarda por cada sujeto un diccionario
      con los datos de las series de tiempo para cada estado (N2,POS,PRE) y por cada filtro o altitud (A,B,C) '''

  # se itera sobre los sujetos filtrados
  for sujeto in dictionary:

    # para cada sujeto se crea el diccionario datos_sujeto que almacenara las series de tiempo
    datos_sujeto = {}

    # por cada estado se cargan las series de tiempo que luego se depositan en datos_sujeto
    for etapa in dictionary[sujeto]:
      fdata_etapa = join(home_predata, etapa)
      time_series = pd.read_table(fdata_etapa, delimiter = ' ', decimal = '.', names = ['RR'])

      # Para que queden por cada sujeto los estados N2, POS y PRE
      name_etapa = etapa[len('#####'): len(etapa)-4] 
      datos_sujeto[name_etapa] = time_series['RR'].tolist()

    # finalmente, se guardan los diccionarios para cada sujeto de las series de tiempo
    fdic = join(home_data, filtro + '_' + sujeto + '.npy')
    #np.save(fdic, datos_sujeto)